In [ ]:
!pip install git+https://github.com/amazon-science/ReFinED.git

In [2]:
from refined.inference.processor import Refined
refined = Refined.from_pretrained(
    model_name='wikipedia_model',
    entity_set="wikidata"
)

Some weights of the model checkpoint at /root/.cache/refined/roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at /root/.cache/refined/roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS e

In [3]:
from refined.data_types.base_types import Entity, Span
def spans_to_mind_format(results : Span):
    entity_list = []
    for entity in results:
        if entity.predicted_entity.wikidata_entity_id == None:
            continue
            # An entity has been detected but does not have a wikidata page
        entity_list.append({
            'Label' : entity.predicted_entity.wikipedia_entity_title,
            'Type': entity.coarse_mention_type,
            'WikidataId': entity.predicted_entity.wikidata_entity_id,
            'Confidence': entity.candidate_entities[0][1],
            'OccurrenceOffsets': [entity.start],
            'SurfaceForms': [entity.text]
        })
    return entity_list

In [5]:
from transformers import M2M100ForConditionalGeneration
from utils.tokenization_small100 import SMALL100Tokenizer
model = M2M100ForConditionalGeneration.from_pretrained("alirezamsh/small100")
tokenizer = SMALL100Tokenizer.from_pretrained("alirezamsh/small100")

Downloading:   0%|          | 0.00/890 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.54M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.82k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'M2M100Tokenizer'. 
The class this function is called from is 'SMALL100Tokenizer'.


In [7]:
!pip install cherrypy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.4/348.4 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.6/100.6 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 13.4 MB/s eta 0:00:00


In [2]:
import cherrypy
import json

In [3]:
class EntityLinker(object):
    exposed = True
    def GET(self, *path, **query):
        no_text = query.get('text', None)
        tokenizer.tgt_lang = "en"
        encoded_no = tokenizer(no_text, return_tensors="pt")
        generated_tokens = model.generate(**encoded_no)
        results = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        en_text = results[0]
        list_entity = spans_to_mind_format(refined.process_text(title))
        list_entity_json = json.dumps(list_entity)
        return list_entity_json

In [ ]:
conf = {
    '/' : {'request.dispatch' : cherrypy.dispatch.MethodDispatcher() } # configuration of the path to address
}
cherrypy.tree.mount(EntityLinker(), '/linker', conf) # one for each object + passing the configuration

cherrypy.config.update({'server.socket_host': '0.0.0.0'})
cherrypy.config.update({'server.socket_port': 8099})

cherrypy.engine.start()


cherrypy.engine.block()

[02/Feb/2023:16:14:21] ENGINE Bus STARTING
[02/Feb/2023:16:14:21] ENGINE Started monitor thread 'Autoreloader'.
[02/Feb/2023:16:14:21] ENGINE Serving on http://0.0.0.0:8099
[02/Feb/2023:16:14:21] ENGINE Bus STARTED
